
# Stock Analysis using OpenAI Functions and LangChain Multi Agent

#### Developed By: Manaranjan Pradhan
#### www.manaranjanp.com

*This Jupyter notebook is confidential and proprietary to Manaranjan Pradhan. It is intended solely for authorized training purposes. Unauthorized distribution, sharing, or reproduction of this notebook or its contents is strictly prohibited. This material is for personal learning within the training program only and may not be used for commercial purposes or shared with others. Unauthorized use may result in disciplinary action or legal consequences. If you have received this notebook without authorization, please contact manaranjan@gmail.com immediately and delete all copies.*

**Diclaimer**: This is only a demo notebook of stocks analysis. The recommendations or advise may not be used for actual invesment decision.

In [1]:
!pip install -q yfinance langchain openai langchain_openai langchainhub langsmith httpx==0.27.2 langchain_groq langgraph


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.2 MB/s eta 0:00:00


In [2]:
import openai
import json
import os
import numpy as np
import yfinance as yf
from langchain_openai import ChatOpenAI
from langchain import hub
from getpass import getpass
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

## Defining a custom method for obtaining stats of a stock

**yfinance** is an open source library to access the financial data available on Yahoo Finance, which offers an excellent range of market data on stocks, bonds and currencies along with market news, reports and analysis.

- Define a function which returns total gains, average and standard deviation of daily prices changes of stocks for a specified number of months

In [3]:
@tool
def get_stock_stats(ticker: str):

    '''
        input string ticker is The correct ticker symbol for the stock in Nation Stock Exchange (NSE) India.
    '''

    # Get ticker name correctly
    msft = yf.Ticker(ticker.split(".")[0] + ".NS")

    # get historical market data
    hist = msft.history(period = str(3) + "mo" )

    # Compute the market data
    hist['daily_changes']  = (hist['Close'] - hist['Open']) * 100 / hist['Open']

    # Compute different statistics
    total_gain = (hist.iloc[-1]["Close"] - hist.iloc[0]["Open"] ) * 100 / hist.iloc[0]["Open"]
    avg_daily_changes = np.mean(hist['daily_changes'])
    std_daily_changes = np.std(hist['daily_changes'])

    stock_stats = {'total_gain_in_percentage': round(total_gain, 3),
                   'average_daily_changes_in_percentage': round(avg_daily_changes, 3),
                   'std_daily_changes_in_percentage': round(std_daily_changes, 3)}

    return json.dumps(stock_stats)

In [4]:
get_stock_stats("TATAPOWER")

<ipython-input-4-901cac82d0f9>:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_stock_stats("TATAPOWER")


'{"total_gain_in_percentage": -18.165, "average_daily_changes_in_percentage": -0.422, "std_daily_changes_in_percentage": 1.783}'

## Configuring the function as a tool

## Configuring tool as OpenAI function

In [5]:
#os.environ["OPENAI_API_KEY"] = getpass('OPENAI_API_KEY')
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [ ]:
#os.environ["LANGCHAIN_API_KEY"] = getpass("Langchain API Key:")

In [6]:
#llm = ChatOpenAI(model_name='gpt-4o-2024-08-06',
#             temperature=0.0,
#             max_tokens = 2048)

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_retries=2,
)

In [7]:
tools = [get_stock_stats]

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)

In [9]:
resp = agent_executor.invoke({"input": "How to find out if a stock is highly volatile?"})
print(resp)



> Entering new AgentExecutor chain...

Invoking: `get_stock_stats` with `{'ticker': 'SBIN'}`


{"total_gain_in_percentage": -15.718, "average_daily_changes_in_percentage": -0.374, "std_daily_changes_in_percentage": 1.309}To determine if a stock is highly volatile, you can use the standard deviation of its daily price changes. A higher standard deviation indicates higher volatility. In this case, the standard deviation of daily price changes for the stock with the ticker symbol "SBIN" is 1.309%, which can be considered relatively low. However, please note that volatility can change over time and may be affected by various market and economic factors. It's always a good idea to consult with a financial advisor or conduct your own research before making investment decisions.

> Finished chain.
{'input': 'How to find out if a stock is highly volatile?', 'output': 'To determine if a stock is highly volatile, you can use the standard deviation of its daily price changes. A higher standard 

In [10]:
resp = agent_executor.invoke({"input": "What is average return for the stock tatapower?"})
print(resp)



> Entering new AgentExecutor chain...

Invoking: `get_stock_stats` with `{'ticker': 'TATAPOWER'}`


{"total_gain_in_percentage": -18.269, "average_daily_changes_in_percentage": -0.424, "std_daily_changes_in_percentage": 1.782}The average return for the stock TATAPOWER is -0.424% per day.

> Finished chain.
{'input': 'What is average return for the stock tatapower?', 'output': 'The average return for the stock TATAPOWER is -0.424% per day.'}


## OpenAI Multi Function Agent

In [11]:
response = agent_executor.invoke( {"input":
    "Between tatapower and adanipower in NSE India, which stock had \
    observerd higher volatility in last one month? Explain the steps for your response. \
    The correct ticket symbol for NSE should be generated from the company names and passed as parameter to tools as necessary."}
)



> Entering new AgentExecutor chain...

Invoking: `get_stock_stats` with `{'ticker': 'TATAPOWER'}`
responded: To determine which stock had higher volatility in the last one month between Tata Power and Adani Power in NSE India, I will follow these steps:

1. Identify the correct ticker symbols for Tata Power and Adani Power in NSE India. The ticker symbol for Tata Power is "TATAPOWER" and for Adani Power is "ADANIPOWER".
2. Use the 'get_stock_stats' function to retrieve the stock statistics for both companies.
3. Compare the volatility of the two stocks based on the retrieved statistics.



{"total_gain_in_percentage": -18.453, "average_daily_changes_in_percentage": -0.428, "std_daily_changes_in_percentage": 1.782}
Invoking: `get_stock_stats` with `{'ticker': 'ADANIPOWER'}`
responded: To determine which stock had higher volatility in the last one month between Tata Power and Adani Power in NSE India, I will follow these steps:

1. Identify the correct ticker symbols for Tata Power and

In [12]:
response

{'input': 'Between tatapower and adanipower in NSE India, which stock had     observerd higher volatility in last one month? Explain the steps for your response.     The correct ticket symbol for NSE should be generated from the company names and passed as parameter to tools as necessary.',
 'output': 'To determine which stock had higher volatility in the last one month between TATAPOWER and ADANIPOWER, we need to compare their standard deviations of daily changes in percentage.\n\nThe standard deviation of daily changes in percentage for TATAPOWER is 1.782, while for ADANIPOWER it is 3.049. \n\nSince 3.049 is greater than 1.782, ADANIPOWER had higher volatility in the last one month.'}

In [13]:
import textwrap

In [14]:
wrapped_text = textwrap.fill(response['output'],
                             width=120,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

To determine which stock had higher volatility in the last one month between TATAPOWER and ADANIPOWER, we need to
compare their standard deviations of daily changes in percentage.

The standard deviation of daily changes in percentage
for TATAPOWER is 1.782, while for ADANIPOWER it is 3.049. 

Since 3.049 is greater than 1.782, ADANIPOWER had higher
volatility in the last one month.
